**Навигация по уроку**

1. [Библиотеки визуализации данных в Python](https://colab.research.google.com/drive/1IbLhzlqiJhhPAErmdQ9wuIUdDaSRlNUF)
2. [Практика](https://colab.research.google.com/drive/1jukOs54u301WtyQS-SbRxWxDstTCjEBq)
3. Домашняя работа

Используя датасет о стоимости акций Сбербанка с 01.01.2013 года:

https://storage.yandexcloud.net/academy.ai/SBER.csv

визуализируйте индикатор "Полосы Боллинджера", проанализируйте график, и предложите вариант торговли акциями Сбербанка с помощью этого инструмента.

**Подсказка.**
Индикатор выглядит как полоса из трех линий:

* линия посередине — это простая скользящая средняя (SMA) с периодом `ma_size`, обычно около 20 дней;

* верхняя и нижняя линии (BB) — построены на основе SMА, но к нему добавленно стреднеквадратичное отклонение:

```
 SMA = data['close'].rolling(ma_size).mean()
 BB_UP = SMA + data['close'].rolling(ma_size).std() * bol_size
 BB_DOWN = SMA - data['close'].rolling(ma_size).std() * bol_size
   
```

где bol_size - ширина коридора, подбирается по графику. Выберите такое его значение, чтобы по графику можно было принимать торговые решения.

In [1]:
import os
import numpy as np
import pandas as pd

import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [2]:
!wget https://storage.yandexcloud.net/academy.ai/SBER.csv


--2024-01-01 16:06:04--  https://storage.yandexcloud.net/academy.ai/SBER.csv
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180264 (176K) [text/csv]
Saving to: ‘SBER.csv’

SBER.csv            100%[===================>] 176.04K   465KB/s    in 0.4s    

2024-01-01 16:06:05 (465 KB/s) - ‘SBER.csv’ saved [180264/180264]



In [3]:
df = pd.read_csv("./SBER.csv")

In [4]:
df.head()

,DATE;OPEN;HIGH;LOW;CLOSE;VOL
0,20130108;96.5000000;98.5000000;96.1200000;98.3...
1,20130109;98.4100000;98.6500000;97.8100000;98.2...
2,20130110;98.3500000;98.5500000;97.9600000;98.4...
3,20130111;98.8000000;99.7600000;98.4800000;99.5...
4,20130114;99.7200000;101.2700000;99.5700000;100...


In [5]:
df['DATE'], df['OPEN'], df['HIGH'], df['LOW'], df['CLOSE'], df['VOL'] = df['DATE;OPEN;HIGH;LOW;CLOSE;VOL'].str.split(';').str

<ipython-input-5-f47e058cb7bb>:1: FutureWarning:

Columnar iteration over characters will be deprecated in future releases.



In [6]:
df.drop(columns=['DATE;OPEN;HIGH;LOW;CLOSE;VOL'], inplace=True)

In [7]:
df['DATE'] = pd.to_datetime(df['DATE'], format='%Y%m%d')

In [8]:
columns_to_convert = ['OPEN', 'HIGH', 'LOW', 'CLOSE']

In [9]:
df[columns_to_convert] = df[columns_to_convert].astype(float)

In [10]:
df['VOL'] = df['VOL'].astype('int64')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2718 entries, 0 to 2717
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   DATE    2718 non-null   datetime64[ns]
 1   OPEN    2718 non-null   float64       
 2   HIGH    2718 non-null   float64       
 3   LOW     2718 non-null   float64       
 4   CLOSE   2718 non-null   float64       
 5   VOL     2718 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 127.5 KB


In [ ]:
df.index = range(len(df))

In [12]:
fig = go.Figure([go.Scatter(x=df['DATE'], y=df['CLOSE'])])
fig.show()

In [13]:
fig.update_xaxes(rangeslider_visible=True, rangeselector=dict(
    buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

In [15]:
ma_size = 20  # Период скользящей средней
std_dev_multiplier = 2  # Множитель стандартного отклонения для полос Боллинджера

# Вычисление скользящей средней и полос Боллинджера
df['SMA'] = df['CLOSE'].rolling(window=ma_size).mean()
df['Upper Band'] = df['SMA'] + (std_dev_multiplier * df['CLOSE'].rolling(window=ma_size).std())
df['Lower Band'] = df['SMA'] - (std_dev_multiplier * df['CLOSE'].rolling(window=ma_size).std())

# Создание графика
fig = go.Figure()

# Цены
fig.add_trace(go.Scatter(x=df['DATE'], y=df['CLOSE'], mode='lines', name='Prices'))

# Скользящая средняя
fig.add_trace(go.Scatter(x=df['DATE'], y=df['SMA'], mode='lines', name=f'SMA-{ma_size}'))

# Полосы Боллинджера
fig.add_trace(go.Scatter(x=df['DATE'], y=df['Upper Band'], mode='lines', name='Upper Band'))
fig.add_trace(go.Scatter(x=df['DATE'], y=df['Lower Band'], mode='lines', name='Lower Band'))

# Настройка макета
fig.update_layout(title='индикатор "Полосы Боллинджера"',
                  xaxis_title='Дата',
                  yaxis_title='Цена',
                  legend=dict(x=0, y=1, traceorder='normal'))
fig.update_xaxes(rangeslider_visible=True, rangeselector=dict(
    buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

# Отображение графика
fig.show()